In [ ]:
import numpy as np
from scipy.signal import cont2discrete, lfilter
import matplotlib.pyplot as plt

import numpy as np
from scipy.signal import cont2discrete, lfilter

import numpy as np
from scipy.signal import cont2discrete, lfilter

def synthesize_shock_pulse(
    srs_spec_hz_g,
    fs=20480,
    duration=0.25,
    q=10.0,
    freqs_per_octave=12,
    n_trials=120,
    inner_iters=18,
    nm_choices=(5, 7, 9, 11, 13),
    rng_seed=None,
    # shock-shaping knobs (both bases):
    t0=0.010,              # main shock start [s]
    tail_span=0.060,       # length of trailing tail [s]
    focus=0.85,            # 0–1: how tightly arrivals cluster near t0
    late_energy_tau=0.050, # window for the time-concentration score [s]
    w_time=0.6,            # weight of time-concentration in winner score
    w_simplicity=0.08,     # weight on zero-crossing count
    clip_scale=(0.25, 4.0),# numerical guard for SRS scaling (not a physical limit)
    # basis selection:
    basis="wavelet",       # "wavelet" or "damped_sine"
    ds_zeta=0.06,          # damping ratio for damped-sine basis (≈ 1/(2Q_ds))
    zero_drift_fix=None    # None | "poly" (optional drift cleanup for damped_sine)
):
    """
    SRS-matching shock synthesis with a 'shocky' time profile.
    Choose basis="wavelet" (compact-support NESC wavelets) or basis="damped_sine"
    (A * e^{-ζ*2πf*t} * sin(2πf*t)), both using iterative SRS scaling.

    Returns
    -------
    t : (N,) time vector [s]
    acc_g : (N,) base motion acceleration [G]
    report : dict with SRS results + shock metrics
    """

    rng = np.random.default_rng(rng_seed)

    # ---------- helpers ----------
    def log_interp(x, xp, fp):
        x = np.asarray(x); xp = np.asarray(xp); fp = np.asarray(fp)
        lx = np.log10(x); lxp = np.log10(xp); lfp = np.log10(fp)
        out = np.interp(lx, lxp, lfp, left=lfp[0], right=lfp[-1])
        return 10**out

    def build_freq_grid(f_lo, f_hi, nper_oct):
        n_oct = np.log2(f_hi / f_lo)
        n_pts = int(np.floor(n_oct * nper_oct)) + 1
        return f_lo * (2.0 ** (np.arange(n_pts) / nper_oct))

    def basis_row_wavelet(t, A, f, delay, Nm):
        # Compact-support (odd Nm>=5) wavelet: zero net v/disp for each component.
        start = delay
        stop = delay + Nm * 0.5 / f
        idx = (t >= start) & (t <= stop)
        z = np.zeros_like(t)
        tau = t[idx] - delay
        z[idx] = A * np.sin(2*np.pi*f * tau / Nm) * np.sin(2*np.pi*f * tau)
        return z

    def basis_row_damped_sine(t, A, f, delay, zeta):
        # A * exp(-zeta*2π f (t-d)) * sin(2π f (t-d)) for t>=delay; 0 otherwise.
        idx = t >= delay
        z = np.zeros_like(t)
        tau = t[idx] - delay
        z[idx] = A * np.exp(-zeta*2*np.pi*f * tau) * np.sin(2*np.pi*f * tau)
        return z

    def srs_accel_abs(acc_g, fs, freqs_hz, q):
        # Absolute-acceleration SRS via bilinear-discretized SDOF filters.
        G2SI = 9.80665
        ydd = np.asarray(acc_g) * G2SI
        zeta = 1.0 / (2.0 * q)
        dt = 1.0 / fs
        out = np.empty_like(freqs_hz, float)
        for i, fn in enumerate(freqs_hz):
            wn = 2*np.pi*fn
            num = [2*zeta*wn, wn**2]
            den = [1.0, 2*zeta*wn, wn**2]
            bz, az, _ = cont2discrete((num, den), dt, method='bilinear')
            b = bz.flatten(); a = az.flatten()
            a_abs = lfilter(b, a, ydd)
            out[i] = np.max(np.abs(a_abs)) / G2SI
        return out

    def clustered_delays(freqs, support_like, duration, t0, tail_span, focus, rng):
        # High-f earlier (near t0), low-f later; small jitter ∝ support.
        order = np.argsort(-freqs)          # descending f -> ranks 0..1
        r = np.empty_like(freqs, float)
        r[order] = np.linspace(0.0, 1.0, freqs.size)
        base = t0 + r * tail_span
        jitter = (rng.random(freqs.size) - 0.5) * (1.0 - focus) * support_like
        delay = np.clip(base + jitter, 0.0, np.maximum(0.0, duration - support_like - 2.0/fs))
        return delay

    def zero_crossings(x):
        s = np.signbit(x)
        return int(np.count_nonzero(s[1:] ^ s[:-1]))

    # ---------- targets ----------
    srs_spec_hz_g = np.asarray(srs_spec_hz_g, float)
    if srs_spec_hz_g.ndim != 2 or srs_spec_hz_g.shape[1] != 2:
        raise ValueError("srs_spec_hz_g must be (M,2) array of (freq_Hz, SRS_G).")
    fmin, fmax = float(srs_spec_hz_g[0,0]), float(srs_spec_hz_g[-1,0])
    freqs = build_freq_grid(fmin, fmax, freqs_per_octave)
    target_srs = log_interp(freqs, srs_spec_hz_g[:,0], srs_spec_hz_g[:,1])

    t = np.arange(int(round(duration*fs))) / fs

    # basis selector
    use_wavelet = (basis.lower() == "wavelet")
    if not use_wavelet and basis.lower() != "damped_sine":
        raise ValueError("basis must be 'wavelet' or 'damped_sine'.")

    # ---------- search (outer trials) ----------
    best = dict(score=np.inf)
    for trial in range(n_trials):
        if use_wavelet:
            Nm = np.asarray([np.random.default_rng(rng.integers(1<<31)).choice(nm_choices) for _ in freqs])
            # support length for delay placement
            support_like = Nm * 0.5 / freqs
        else:
            Nm = None
            # effective support proxy for placing delays (few decay constants)
            support_like = np.minimum(duration, 3.0/(ds_zeta*2*np.pi*freqs))

        # start with positive amplitudes to favor a single dominant lobe
        A = 0.3 * target_srs * (1 + 0.5*(rng.random(freqs.size)-0.5))
        A = np.abs(A)

        delay = clustered_delays(freqs, support_like, duration, t0, tail_span, focus, rng)

        # precompute basis rows (unit amplitude)
        basis_rows = np.empty((freqs.size, t.size), float)
        if use_wavelet:
            for k, (fk, dk, Nk) in enumerate(zip(freqs, delay, Nm)):
                basis_rows[k] = basis_row_wavelet(t, 1.0, fk, dk, Nk)
        else:
            for k, (fk, dk) in enumerate(zip(freqs, delay)):
                basis_rows[k] = basis_row_damped_sine(t, 1.0, fk, dk, ds_zeta)

        # inner loop: SRS amplitude scaling
        for _ in range(inner_iters):
            acc = A @ basis_rows
            achieved = srs_accel_abs(acc, fs, freqs, q)
            scale = np.divide(target_srs, np.maximum(achieved, 1e-12))
            scale = np.clip(scale, *clip_scale)  # stability only
            A *= scale

        # final evaluation for this trial
        acc = A @ basis_rows

        # optional drift cleanup for damped_sine (analysis convenience)
        if (not use_wavelet) and (zero_drift_fix == "poly"):
            # remove tiny linear trend in velocity to suppress residual drift
            G2SI = 9.80665
            v = np.cumsum(acc*G2SI) / fs
            # fit a line to v and subtract its derivative from acceleration
            n = len(v); x = np.arange(n)
            c1, c0 = np.polyfit(x, v, 1)  # v ≈ c1*x + c0
            acc = acc - (c1*G2SI)/fs  # derivative of linear term

        achieved = srs_accel_abs(acc, fs, freqs, q)
        err_db = 20.0*np.log10(np.maximum(achieved,1e-12)/np.maximum(target_srs,1e-12))
        max_abs_err_db = float(np.max(np.abs(err_db)))

        # shock-aware score (analysis mode): SRS error + time-concentration + simplicity
        win_lo = int(np.floor(t0*fs))
        win_hi = int(np.floor(min(duration, t0+late_energy_tau)*fs))
        e_total = float(np.sum(acc**2) + 1e-18)
        e_focus = float(np.sum(acc[win_lo:win_hi]**2))
        time_penalty = 1.0 - (e_focus/e_total)       # smaller is better
        zc = zero_crossings(acc)
        simplicity_penalty = zc / max(1, len(acc)//8)
        score = max_abs_err_db + w_time*(10.0*time_penalty) + w_simplicity*(10.0*simplicity_penalty)

        if score < best.get("score", np.inf) or (
            np.isclose(score, best.get("score", np.inf)) and max_abs_err_db < best.get("max_abs_err_db", np.inf)
        ):
            G2SI = 9.80665
            v = np.cumsum(acc*G2SI) / fs
            d = np.cumsum(v) / fs
            best.update(dict(
                score=score,
                max_abs_err_db=max_abs_err_db,
                acc=acc.copy(),
                achieved=achieved.copy(),
                trial=trial,
                time_focus_ratio=e_focus/e_total,
                zero_crossings=int(zc),
                peakA=float(np.max(np.abs(acc))),
                peakV=float(np.max(np.abs(v))),
                peakD=float(np.max(np.abs(d))),
                delays=delay.copy(),
                Nm=None if Nm is None else Nm.copy(),
                basis=basis.lower()
            ))

    report = dict(
        freqs_hz=freqs,
        target_srs_g=target_srs,
        achieved_srs_g=best["achieved"],
        max_abs_error_db=best["max_abs_err_db"],
        time_focus_ratio=float(best["time_focus_ratio"]),
        zero_crossings=int(best["zero_crossings"]),
        peak_accel_g=float(best["peakA"]),
        peak_velocity_si=float(best["peakV"]),
        peak_displacement_si=float(best["peakD"]),
        winner_trial=int(best["trial"]),
        q=float(q), fs=float(fs),
        t0=float(t0), tail_span=float(tail_span),
        basis=best["basis"]
    )
    return t, best["acc"], report





In [ ]:
# Run the example and create plots
# Spec like the example on p.7: (10 Hz, 9.4 g) → (80 Hz, 75 g) → (2000 Hz, 75 g)
srs_spec = np.array([[10.0, 9.4],
                     [80.0, 75.0],
                     [2000.0, 75.0]])

print("Synthesizing shock signal to match SRS specification...")
t, acc_g, info = synthesize_shock_pulse(
    srs_spec, fs=20480, duration=0.25, q=10, freqs_per_octave=12,
    n_trials=60, inner_iters=16, rng_seed=3, basis="damped_sine"
)

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Time history of synthesized acceleration
ax1.plot(t * 1000, acc_g, 'b-', linewidth=1)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Acceleration (G)')
ax1.set_title(f'Synthesized Shock Acceleration Signal\n'
              f'Peak: {info["peak_accel_g"]:.1f} G, Duration: {0.25*1000:.0f} ms')
ax1.grid(True, alpha=0.3)

# Plot 2: SRS comparison (target vs achieved)
ax2.loglog(srs_spec[:, 0], srs_spec[:, 1], 'ro-', linewidth=2, markersize=8, 
           label='Target SRS Spec', markerfacecolor='white', markeredgewidth=2)
ax2.loglog(info["freqs_hz"], info["target_srs_g"], 'r--', alpha=0.7, 
           label='Target SRS (interpolated)')
ax2.loglog(info["freqs_hz"], info["achieved_srs_g"], 'b-', linewidth=2, 
           label='Achieved SRS')

ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('SRS Acceleration (G)')
ax2.set_title(f'Shock Response Spectrum Comparison\n'
              f'Q = {info["q"]:.0f}, Max Error: {info["max_abs_error_db"]:.2f} dB, '
              f'Winner Trial: {info["winner_trial"]}')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim([srs_spec[0, 0] * 0.8, srs_spec[-1, 0] * 1.2])

plt.tight_layout()
plt.show()

# Print summary information
print(f"\nSynthesis Summary:")
print(f"- Duration: {0.25*1000:.0f} ms")
print(f"- Sample rate: {info['fs']:.0f} Hz")
print(f"- Q factor: {info['q']:.0f}")
print(f"- Peak acceleration: {info['peak_accel_g']:.1f} G")
print(f"- Max SRS error: {info['max_abs_error_db']:.2f} dB")
print(f"- Winning trial: {info['winner_trial']} (out of 60)")

In [ ]:
# Step 1: Generate a half-sine pulse for SRS analysis and synthesis comparison
print("Complete Half-Sine Pulse to SRS Synthesis Workflow")
print("=" * 55)

# Import required functions
import time
from mdof_utilities import generate_half_sine_pulse, shock_response_spectrum

print("\n1. Generating original half-sine pulse...")

# Define half-sine pulse parameters
pulse_amplitude = 50    # 50g peak acceleration
pulse_duration = 0.02  # 2 ms duration
total_time = 0.1       # 100 ms total signal length
sample_rate = 20480    # High sample rate for accuracy

# Generate the half-sine pulse using the utility function
t_original, a_original = generate_half_sine_pulse(
    amplitude=pulse_amplitude,
    duration=pulse_duration, 
    total_time=total_time,
    sample_rate=sample_rate
)

print(f"   - Pulse amplitude: {pulse_amplitude} g")
print(f"   - Pulse duration: {pulse_duration*1000:.1f} ms") 
print(f"   - Total signal length: {total_time*1000:.0f} ms")
print(f"   - Sample rate: {sample_rate} Hz")
print(f"   - Number of samples: {len(t_original)}")

# Plot the generated pulse
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(t_original * 1000, a_original, 'b-', linewidth=2)
plt.xlabel('Time (ms)')
plt.ylabel('Acceleration (g)')
plt.title(f'Generated Half-Sine Pulse\n{pulse_amplitude}g peak, {pulse_duration*1000:.1f}ms duration')
plt.grid(True, alpha=0.3)

# Zoom view of just the pulse
pulse_end_time = pulse_duration * 2  # Show twice the pulse duration
pulse_mask = t_original <= pulse_end_time
plt.subplot(1, 2, 2)
plt.plot(t_original[pulse_mask] * 1000, a_original[pulse_mask], 'r-', linewidth=2)
plt.xlabel('Time (ms)')
plt.ylabel('Acceleration (g)')
plt.title(f'Pulse Detail (First {pulse_end_time*1000:.1f}ms)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Step 2: Compute SRS of the half-sine pulse
print("\n2. Computing SRS of the original pulse...")

# Define frequency range for SRS calculation
freq_min = 10.0     # 10 Hz minimum
freq_max = 1000.0   # 1000 Hz maximum
n_freq = 25         # 25 frequency points (optimized for speed)

freq_range = np.logspace(np.log10(freq_min), np.log10(freq_max), n_freq)

# Calculate SRS with optimized settings
start_time = time.time()
srs_original = shock_response_spectrum(
    t_original, a_original, 
    freq_range=freq_range,
    damping_ratio=0.05,  # Q = 10 (5% damping)
    speed_level="optimal"
)
srs_time = time.time() - start_time

print(f"   - Frequency range: {freq_min:.0f} to {freq_max:.0f} Hz")
print(f"   - Number of frequencies: {n_freq}")
print(f"   - SRS calculation time: {srs_time:.2f} seconds")
print(f"   - Peak SRS: {np.max(srs_original):.1f} g at {freq_range[np.argmax(srs_original)]:.1f} Hz")

# Plot the SRS
plt.figure(figsize=(12, 6))
plt.loglog(freq_range, srs_original, 'ro-', linewidth=2, markersize=6, 
           markerfacecolor='white', markeredgewidth=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('SRS Acceleration (g)')
plt.title(f'SRS of Original Half-Sine Pulse (Q=10)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Step 3: Use the SRS as a target specification for synthesis
print("\n3. Creating target SRS specification...")

# Convert the computed SRS into the format expected by synthesize_shock_srs
# Format: [[freq1, srs1], [freq2, srs2], ...]
srs_target_spec = np.column_stack([freq_range, srs_original])

print(f"   - Target specification created with {len(srs_target_spec)} points")
print(f"   - Frequency range: {srs_target_spec[0,0]:.1f} to {srs_target_spec[-1,0]:.1f} Hz")
print(f"   - SRS range: {srs_target_spec[:,1].min():.1f} to {srs_target_spec[:,1].max():.1f} g")

# Step 4: Synthesize a signal that matches this SRS
print("\n4. Synthesizing signal to match the target SRS...")

synthesis_params = {
    'fs': 20480,           # Higher sample rate for better accuracy
    'duration': 0.25,      # 250 ms duration
    'q': 10,              # Q = 10 (same as SRS calculation)
    'freqs_per_octave': 12, # Reduced for speed
    'n_trials': 300,        # Moderate number of trials
    'inner_iters': 10,     # Moderate iterations
    'rng_seed': 41         # For reproducibility
}

print(f"   - Sample rate: {synthesis_params['fs']} Hz")
print(f"   - Duration: {synthesis_params['duration']*1000:.0f} ms")
print(f"   - Trials: {synthesis_params['n_trials']}")

start_time = time.time()
t_synth, acc_synth, info_synth = synthesize_shock_pulse(
    srs_target_spec, **synthesis_params
)
synth_time = time.time() - start_time

print(f"   - Synthesis completed in {synth_time:.1f} seconds")
print(f"   - Peak synthesized acceleration: {info_synth['peak_accel_g']:.1f} g")
print(f"   - SRS matching error: {info_synth['max_abs_error_db']:.2f} dB")
print(f"   - Winner trial: {info_synth['winner_trial']}/{synthesis_params['n_trials']}")

In [ ]:
# Step 5: Compare original pulse with synthesized signal
print("\n5. Comparing results...")

# Create comprehensive comparison plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Time domain comparison (first 50 ms)
time_limit = 0.05  # 50 ms
mask_orig = t_original <= time_limit
mask_synth = t_synth <= time_limit

axes[0,0].plot(t_original[mask_orig] * 1000, a_original[mask_orig], 
               'b-', linewidth=2, label=f'Original Half-Sine ({pulse_amplitude}g peak)')
axes[0,0].plot(t_synth[mask_synth] * 1000, acc_synth[mask_synth], 
               'r-', linewidth=1, alpha=0.8, 
               label=f'Synthesized ({info_synth["peak_accel_g"]:.0f}g peak)')
axes[0,0].set_xlabel('Time (ms)')
axes[0,0].set_ylabel('Acceleration (g)')
axes[0,0].set_title('Time Domain Comparison (First 50ms)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: SRS comparison
axes[0,1].loglog(freq_range, srs_original, 'bo-', linewidth=2, markersize=6,
                 markerfacecolor='white', markeredgewidth=2, label='Target SRS (Original)')
axes[0,1].loglog(info_synth["freqs_hz"], info_synth["target_srs_g"], 'b--', 
                 alpha=0.7, label='Target SRS (Interpolated)')
axes[0,1].loglog(info_synth["freqs_hz"], info_synth["achieved_srs_g"], 'r-', 
                 linewidth=2, label='Achieved SRS (Synthesized)')
axes[0,1].set_xlabel('Frequency (Hz)')
axes[0,1].set_ylabel('SRS Acceleration (g)')
axes[0,1].set_title(f'SRS Comparison (Max Error: {info_synth["max_abs_error_db"]:.2f} dB)')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Plot 3: SRS Error
srs_error_db = 20 * np.log10(info_synth["achieved_srs_g"] / info_synth["target_srs_g"])
axes[1,0].semilogx(info_synth["freqs_hz"], srs_error_db, 'g-', linewidth=2)
axes[1,0].axhline(y=0, color='k', linestyle='--', alpha=0.5)
axes[1,0].axhline(y=1, color='r', linestyle=':', alpha=0.5, label='±1 dB')
axes[1,0].axhline(y=-1, color='r', linestyle=':', alpha=0.5)
axes[1,0].set_xlabel('Frequency (Hz)')
axes[1,0].set_ylabel('SRS Error (dB)')
axes[1,0].set_title('SRS Matching Error vs Frequency')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Full time domain signals
axes[1,1].plot(t_original * 1000, a_original, 'b-', linewidth=1.5, 
               label=f'Original Half-Sine ({pulse_duration*1000:.1f}ms)')
axes[1,1].plot(t_synth * 1000, acc_synth, 'r-', linewidth=1, alpha=0.8,
               label=f'Synthesized ({synthesis_params["duration"]*1000:.0f}ms)')
axes[1,1].set_xlabel('Time (ms)')
axes[1,1].set_ylabel('Acceleration (g)')
axes[1,1].set_title('Complete Time History Comparison')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n" + "="*60)
print("SYNTHESIS SUMMARY")
print("="*60)
print(f"Original pulse:")
print(f"  - Type: Half-sine")
print(f"  - Peak: {pulse_amplitude} g")
print(f"  - Duration: {pulse_duration*1000:.1f} ms")
print(f"  - Peak SRS: {np.max(srs_original):.1f} g")

print(f"\nSynthesized signal:")
print(f"  - Peak: {info_synth['peak_accel_g']:.1f} g")
print(f"  - Duration: {synthesis_params['duration']*1000:.0f} ms")  
print(f"  - Peak SRS: {np.max(info_synth['achieved_srs_g']):.1f} g")
print(f"  - SRS matching error: {info_synth['max_abs_error_db']:.2f} dB")
print(f"  - Synthesis time: {synth_time:.1f} seconds")

print(f"\nConclusion:")
peak_ratio = info_synth['peak_accel_g'] / pulse_amplitude
print(f"  - Peak amplitude ratio: {peak_ratio:.2f}x")
if info_synth['max_abs_error_db'] < 1.0:
    print(f"  - ✅ Excellent SRS match (< 1 dB error)")
elif info_synth['max_abs_error_db'] < 3.0:
    print(f"  - ✅ Good SRS match (< 3 dB error)")
else:
    print(f"  - ⚠️ Moderate SRS match ({info_synth['max_abs_error_db']:.1f} dB error)")

print(f"  - The synthesized signal successfully reproduces the SRS characteristics")
print(f"    of the original half-sine pulse using a complex wavelet series.")

In [ ]:
# Performance and Method Comparison: Wavelet vs Damped Sine Basis
print("SYNTHESIS BASIS COMPARISON: WAVELET vs DAMPED SINE")
print("=" * 60)

# Common parameters for fair comparison
common_params = {
    'fs': 20480,
    'duration': 0.25,
    'q': 10,
    'freqs_per_octave': 8,
    'n_trials': 25,
    'inner_iters': 12,
    'rng_seed': 42,
    # Shock-shaping parameters
    't0': 0.010,
    'tail_span': 0.060,
    'focus': 0.85,
    'late_energy_tau': 0.050,
    'w_time': 0.6,
    'w_simplicity': 0.08
}

print("\n1. Testing Wavelet basis (NESC wavelets)...")
start_time = time.time()
t_wavelet, acc_wavelet, info_wavelet = synthesize_shock_pulse(
    srs_target_spec,
    basis="wavelet",
    **common_params
)
time_wavelet = time.time() - start_time

print(f"   - Computation time: {time_wavelet:.2f} seconds")
print(f"   - Peak acceleration: {info_wavelet['peak_accel_g']:.1f} g")
print(f"   - SRS matching error: {info_wavelet['max_abs_error_db']:.2f} dB")
print(f"   - Time concentration score: {info_wavelet.get('time_concentration_score', 'N/A')}")
print(f"   - Simplicity score: {info_wavelet.get('simplicity_score', 'N/A')}")

print("\n2. Testing Damped sine basis...")
start_time = time.time()
t_damped, acc_damped, info_damped = synthesize_shock_pulse(
    srs_target_spec,
    basis="damped_sine",
    ds_zeta=0.06,  # Damping ratio for damped sines (≈ 1/(2*Q))
    zero_drift_fix="poly",  # Optional drift cleanup
    **common_params
)
time_damped = time.time() - start_time

print(f"   - Computation time: {time_damped:.2f} seconds")
print(f"   - Peak acceleration: {info_damped['peak_accel_g']:.1f} g")
print(f"   - SRS matching error: {info_damped['max_abs_error_db']:.2f} dB")
print(f"   - Time concentration score: {info_damped.get('time_concentration_score', 'N/A')}")
print(f"   - Simplicity score: {info_damped.get('simplicity_score', 'N/A')}")
print(f"   - Damped sine damping ratio: 0.06 (Q ≈ 8.3)")

# Quick comparison summary
print(f"\n3. Quick Comparison:")
speed_improvement = ((time_wavelet - time_damped) / time_wavelet) * 100
accuracy_diff = info_wavelet['max_abs_error_db'] - info_damped['max_abs_error_db']
peak_ratio_w = info_wavelet['peak_accel_g'] / pulse_amplitude
peak_ratio_d = info_damped['peak_accel_g'] / pulse_amplitude

print(f"   - Speed difference: {speed_improvement:+.1f}% (+ means damped sine faster)")
print(f"   - Accuracy difference: {accuracy_diff:+.2f} dB (+ means wavelet more accurate)")
print(f"   - Peak ratios: Wavelet {peak_ratio_w:.2f}x, Damped Sine {peak_ratio_d:.2f}x original")

if abs(accuracy_diff) < 0.5:
    print(f"   - Both methods achieve similar SRS accuracy")
elif accuracy_diff > 0:
    print(f"   - Wavelet method is more accurate by {accuracy_diff:.1f} dB")
else:
    print(f"   - Damped sine method is more accurate by {abs(accuracy_diff):.1f} dB")

In [ ]:
# Detailed Comparison Plots and Analysis
print("\n4. Detailed Performance Analysis...")

# Create comprehensive comparison figure
fig, axes = plt.subplots(3, 2, figsize=(16, 15))

# Plot 1: Time domain comparison (focus on shock region)
shock_time = 0.08  # Focus on first 80ms where main shock occurs
mask_w = t_wavelet <= shock_time
mask_d = t_damped <= shock_time
mask_o = t_original <= shock_time

axes[0,0].plot(t_original[mask_o] * 1000, a_original[mask_o], 'k-', linewidth=2, 
               label='Original Half-Sine', alpha=0.8)
axes[0,0].plot(t_wavelet[mask_w] * 1000, acc_wavelet[mask_w], 'b-', linewidth=1.5, 
               label=f'Wavelet Basis (Peak: {info_wavelet["peak_accel_g"]:.0f}g)', alpha=0.8)
axes[0,0].plot(t_damped[mask_d] * 1000, acc_damped[mask_d], 'r-', linewidth=1.5,
               label=f'Damped Sine Basis (Peak: {info_damped["peak_accel_g"]:.0f}g)', alpha=0.8)
axes[0,0].set_xlabel('Time (ms)')
axes[0,0].set_ylabel('Acceleration (g)')
axes[0,0].set_title('Time Domain Comparison (Main Shock Region)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: SRS comparison
axes[0,1].loglog(freq_range, srs_original, 'ko-', linewidth=2, markersize=6,
                 markerfacecolor='white', markeredgewidth=2, label='Target SRS (Original)')
axes[0,1].loglog(info_wavelet["freqs_hz"], info_wavelet["achieved_srs_g"], 'b-', 
                 linewidth=2, label=f'Wavelet (±{info_wavelet["max_abs_error_db"]:.2f} dB)')
axes[0,1].loglog(info_damped["freqs_hz"], info_damped["achieved_srs_g"], 'r-', 
                 linewidth=2, label=f'Damped Sine (±{info_damped["max_abs_error_db"]:.2f} dB)')
axes[0,1].set_xlabel('Frequency (Hz)')
axes[0,1].set_ylabel('SRS Acceleration (g)')
axes[0,1].set_title('SRS Matching Comparison')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Plot 3: SRS Error comparison
srs_err_wavelet = 20 * np.log10(info_wavelet["achieved_srs_g"] / info_wavelet["target_srs_g"])
srs_err_damped = 20 * np.log10(info_damped["achieved_srs_g"] / info_damped["target_srs_g"])

axes[1,0].semilogx(info_wavelet["freqs_hz"], srs_err_wavelet, 'b-', linewidth=2, 
                   label='Wavelet Basis')
axes[1,0].semilogx(info_damped["freqs_hz"], srs_err_damped, 'r-', linewidth=2, 
                   label='Damped Sine Basis')
axes[1,0].axhline(y=0, color='k', linestyle='--', alpha=0.5)
axes[1,0].axhline(y=1, color='gray', linestyle=':', alpha=0.5, label='±1 dB')
axes[1,0].axhline(y=-1, color='gray', linestyle=':', alpha=0.5)
axes[1,0].set_xlabel('Frequency (Hz)')
axes[1,0].set_ylabel('SRS Error (dB)')
axes[1,0].set_title('SRS Matching Error vs Frequency')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)
axes[1,0].set_ylim([-8, 8])

# Plot 4: Frequency content comparison (FFT)
from scipy.fft import fft, fftfreq
n_fft = len(t_wavelet)
freqs_fft = fftfreq(n_fft, 1/20480)[:n_fft//2]

fft_wavelet = np.abs(fft(acc_wavelet)[:n_fft//2])
fft_damped = np.abs(fft(acc_damped)[:n_fft//2]) 
fft_original = np.abs(fft(a_original)[:len(a_original)//2])
freqs_orig = fftfreq(len(a_original), 1/sample_rate)[:len(a_original)//2]

axes[1,1].loglog(freqs_orig, fft_original, 'k-', linewidth=2, alpha=0.7, label='Original')
axes[1,1].loglog(freqs_fft, fft_wavelet, 'b-', linewidth=1.5, alpha=0.8, label='Wavelet')
axes[1,1].loglog(freqs_fft, fft_damped, 'r-', linewidth=1.5, alpha=0.8, label='Damped Sine')
axes[1,1].set_xlabel('Frequency (Hz)')
axes[1,1].set_ylabel('FFT Magnitude')
axes[1,1].set_title('Frequency Content Comparison')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)
axes[1,1].set_xlim([1, 2000])

# Plot 5: Signal energy concentration (cumulative energy)
def cumulative_energy(signal, dt):
    """Calculate cumulative energy over time"""
    energy = np.cumsum(signal**2) * dt
    return energy / energy[-1]  # Normalize to 1

dt_w = t_wavelet[1] - t_wavelet[0]
dt_d = t_damped[1] - t_damped[0]
dt_o = t_original[1] - t_original[0]

cum_energy_w = cumulative_energy(acc_wavelet, dt_w)
cum_energy_d = cumulative_energy(acc_damped, dt_d)
cum_energy_o = cumulative_energy(a_original, dt_o)

axes[2,0].plot(t_original * 1000, cum_energy_o, 'k-', linewidth=2, label='Original')
axes[2,0].plot(t_wavelet * 1000, cum_energy_w, 'b-', linewidth=2, label='Wavelet')
axes[2,0].plot(t_damped * 1000, cum_energy_d, 'r-', linewidth=2, label='Damped Sine')
axes[2,0].axhline(y=0.9, color='gray', linestyle='--', alpha=0.5, label='90% Energy')
axes[2,0].set_xlabel('Time (ms)')
axes[2,0].set_ylabel('Cumulative Energy Fraction')
axes[2,0].set_title('Energy Concentration Over Time')
axes[2,0].legend()
axes[2,0].grid(True, alpha=0.3)
axes[2,0].set_xlim([0, 100])

# Plot 6: Performance metrics comparison
metrics_names = ['Time (s)', 'Peak (g)', 'SRS Error (dB)']
wavelet_vals = [time_wavelet, info_wavelet['peak_accel_g'], info_wavelet['max_abs_error_db']]
damped_vals = [time_damped, info_damped['peak_accel_g'], info_damped['max_abs_error_db']]

x = np.arange(len(metrics_names))
width = 0.35

bars1 = axes[2,1].bar(x - width/2, wavelet_vals, width, label='Wavelet', color='blue', alpha=0.7)
bars2 = axes[2,1].bar(x + width/2, damped_vals, width, label='Damped Sine', color='red', alpha=0.7)

axes[2,1].set_xlabel('Performance Metrics')
axes[2,1].set_ylabel('Values')
axes[2,1].set_title('Performance Metrics Comparison')
axes[2,1].set_xticks(x)
axes[2,1].set_xticklabels(metrics_names)
axes[2,1].legend()
axes[2,1].grid(True, alpha=0.3)

# Add value labels on bars
for i, (bar1, bar2, val1, val2) in enumerate(zip(bars1, bars2, wavelet_vals, damped_vals)):
    if i == 0:  # Time
        axes[2,1].text(bar1.get_x() + bar1.get_width()/2., bar1.get_height() + 0.02, 
                       f'{val1:.2f}', ha='center', va='bottom', fontsize=9)
        axes[2,1].text(bar2.get_x() + bar2.get_width()/2., bar2.get_height() + 0.02, 
                       f'{val2:.2f}', ha='center', va='bottom', fontsize=9)
    elif i == 1:  # Peak
        axes[2,1].text(bar1.get_x() + bar1.get_width()/2., bar1.get_height() + 1, 
                       f'{val1:.0f}', ha='center', va='bottom', fontsize=9)
        axes[2,1].text(bar2.get_x() + bar2.get_width()/2., bar2.get_height() + 1, 
                       f'{val2:.0f}', ha='center', va='bottom', fontsize=9)
    else:  # Error
        axes[2,1].text(bar1.get_x() + bar1.get_width()/2., bar1.get_height() + 0.05, 
                       f'{val1:.2f}', ha='center', va='bottom', fontsize=9)
        axes[2,1].text(bar2.get_x() + bar2.get_width()/2., bar2.get_height() + 0.05, 
                       f'{val2:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Summary analysis
print(f"\n" + "="*70)
print("BASIS COMPARISON SUMMARY")
print("="*70)
print(f"{'Metric':<25} {'Wavelet Basis':<20} {'Damped Sine Basis':<20} {'Winner':<15}")
print("-"*80)

# Performance comparison
speed_winner = "Damped Sine" if time_damped < time_wavelet else "Wavelet"
accuracy_winner = "Wavelet" if info_wavelet['max_abs_error_db'] < info_damped['max_abs_error_db'] else "Damped Sine"
peak_ratio_w = info_wavelet['peak_accel_g'] / pulse_amplitude
peak_ratio_d = info_damped['peak_accel_g'] / pulse_amplitude
peak_winner = "Damped Sine" if abs(peak_ratio_d - 1.0) < abs(peak_ratio_w - 1.0) else "Wavelet"

print(f"{'Computation Time (s)':<25} {time_wavelet:<20.2f} {time_damped:<20.2f} {speed_winner:<15}")
print(f"{'SRS Error (dB)':<25} {info_wavelet['max_abs_error_db']:<20.2f} {info_damped['max_abs_error_db']:<20.2f} {accuracy_winner:<15}")
print(f"{'Peak Acceleration (g)':<25} {info_wavelet['peak_accel_g']:<20.0f} {info_damped['peak_accel_g']:<20.0f} {peak_winner:<15}")
print(f"{'Peak Ratio to Original':<25} {peak_ratio_w:<20.2f} {peak_ratio_d:<20.2f} {peak_winner:<15}")

print(f"\nBASIS CHARACTERISTICS:")
print(f"• Wavelet Basis: Compact NESC wavelets with zero net impulse")
print(f"• Damped Sine Basis: A*exp(-ζ*2πf*t)*sin(2πf*t) with ζ=0.06")
print(f"• Both use iterative SRS-based amplitude scaling")

print(f"\nCONCLUSIONS:")
print(f"• SRS Accuracy: {accuracy_winner} basis achieves better SRS matching")
print(f"• Computational Speed: {speed_winner} basis is faster") 
print(f"• Peak Amplitude Control: {peak_winner} basis better preserves amplitude relationships")

print(f"\nRECOMMENDATIONS:")
if accuracy_winner == "Wavelet":
    print(f"• For precision applications: Use wavelet basis")
    print(f"• For rapid analysis: Use damped sine basis")
else:
    print(f"• For precision applications: Use damped sine basis")  
    print(f"• For rapid analysis: Use wavelet basis")
    
print(f"• Wavelet basis: More mathematically sophisticated, typically more accurate")
print(f"• Damped sine basis: More physically intuitive, potentially faster convergence")
print("="*70)